## intro to OpenAI Agents SDK

In [34]:
from agents import Agent, Runner
from dotenv import load_dotenv
load_dotenv()

True

In [44]:
agent = Agent(name="Assistant",model="gpt-4o", instructions="Answer as factually as possible")
result = await Runner.run(agent, "siapa direktur utama telkomsel")
print(result.final_output)

Direktur Utama Telkomsel saat ini adalah Hendri Mulya Syam.


In [45]:
from agents import Agent, Runner, function_tool
#function_tool: decorator to register tools
@function_tool
def get_weather(city: str) -> str:
    return f"Sunny in {city}"

#First Create an instance of Agent
agent = Agent(
    name="WeatherBot",
    instructions="Provide weather info",
    tools=[get_weather]
)
# in CLI: result = Runner.run_sync(agent, "What's the weather in Paris?") in python cli
# Call runner.run() to run the agent
result = await Runner.run(agent, "What's the weather in Paris?")
print(result.final_output)

The weather in Paris is currently sunny.


- Use a real weather API (e.g., Open-Meteo)
- Add input/output guardrails using pydantic

In [5]:
#!pip install httpx

In [46]:
# Step 1: Install dependencies if needed
# !pip install openai-agents httpx

# Step 2: Import required modules
import httpx
from agents import Agent, Runner, function_tool, trace

# Step 3: Define the weather tool
@function_tool
async def get_weather(city: str) -> str:
    try:
        # Get coordinates for the city
        geo = httpx.get("https://geocoding-api.open-meteo.com/v1/search", params={"name": city}).json()
        if "results" not in geo or not geo["results"]:
            return "City not found."

        lat = geo["results"][0]["latitude"]
        lon = geo["results"][0]["longitude"]

        # Fetch current weather data
        weather = httpx.get("https://api.open-meteo.com/v1/forecast", params={
            "latitude": lat,
            "longitude": lon,
            "current_weather": True
        }).json()

        temp = weather["current_weather"]["temperature"]
        return f"It is currently {temp}°C in {city}."
    except Exception as e:
        return f"Failed to get weather for {city}: {e}"

# Step 4: Define fallback agent
fallback_agent = Agent(
    name="FallbackAgent",
    instructions="Apologize if the weather data is not available or city is invalid."
)

# Step 5: Define main agent
main_agent = Agent(
    name="WeatherBot",
    instructions="You're a weather assistant. Use tools to answer weather questions. If city is unknown, hand off.",
    tools=[get_weather],
    handoffs={"unknown": fallback_agent}
)

# Step 6: Run agent with tracing (for Jupyter / async environments)
with trace(workflow_name="weather_query"):
    result = await Runner.run(main_agent, "What's the weather in Jakarta?")
    print(result.final_output)


The current temperature in Jakarta is 31.0°C.


In [47]:
from agents import Agent, WebSearchTool, Runner

In [62]:
async def main():
    # 1. Create an agent equipped with web search
    research_agent = Agent(
        name="Research Agent", model="gpt-4o",
        instructions="You're a great search engine and find from official websites",
        tools=[WebSearchTool()]
    )

    # 2. Run a query
    query = "list wifi and internet provider in madiun, all big, medium and small players"
    result = await Runner.run(research_agent, query)

    # 3. Print the full final output with citations
    print("\n🔎 Final Answer:\n")
    print(result.final_output)

await main()


🔎 Final Answer:

Berikut adalah daftar penyedia layanan internet (ISP) di Kota Madiun, mencakup pemain besar, menengah, dan kecil:

**Pemain Besar:**

1. **IndiHome**: Layanan internet berbasis fiber optik dari Telkom Indonesia yang tersedia di Madiun dengan berbagai paket kecepatan hingga 100 Mbps. ([indihomeofficial.id](https://indihomeofficial.id/indihome-madiun/?utm_source=openai))

2. **Biznet**: Menyediakan layanan internet dengan kecepatan hingga 175 Mbps untuk pengguna rumahan dan hingga 10 Gbps untuk perusahaan besar. Biznet telah hadir di Madiun sejak Agustus 2015 dan menawarkan berbagai paket sesuai kebutuhan pelanggan. ([biznet.sg](https://www.biznet.sg/en/company/network/city/place/madiun?utm_source=openai))

3. **MyRepublic**: Memperluas layanannya ke Madiun pada November 2023, menawarkan berbagai paket internet berkecepatan tinggi untuk pelanggan di area tersebut. ([en.wikipedia.org](https://en.wikipedia.org/wiki/MyRepublic?utm_source=openai))

**Pemain Menengah:**

1. 

In [49]:
async def main():
    # 1. Create an agent equipped with web search
    research_agent = Agent(
        name="Research Agent",
        instructions="You are an assistant that returns structured tables based on user queries.",
        tools=[WebSearchTool()]
    )

    # 2. Run a query
    query = "Find internet providers in Madiun (if possible 6 unique provider name)  and return a table with columns: Provider Name, Package, Speed, Price."
    result = await Runner.run(research_agent, query)

    # 3. Print the full final output with citations
    print("\n🔎 Final Answer:\n")
    print(result.final_output)

await main()


🔎 Final Answer:

Here is a table of internet service providers in Madiun, Indonesia, along with their available packages, speeds, and prices:

| Provider Name | Package Name | Speed       | Price (IDR) |
|---------------|--------------|-------------|-------------|
| Biznet        | HOME 0D      | Up to 50 Mbps | 175,000/month |
| Biznet        | HOME 1D      | Up to 150 Mbps | 375,000/month |
| Biznet        | HOME 2D      | Up to 250 Mbps | 575,000/month |
| Biznet        | GAMERS 3D    | Up to 300 Mbps | 700,000/month |
| Biznet        | METRONET 1D  | Up to 100 Mbps | 1,000,000/month |
| Biznet        | METRONET 2D  | Up to 150 Mbps | 1,700,000/month |
| Biznet        | METRONET 3D  | Up to 200 Mbps | 2,700,000/month |
| Biznet        | METRONET 4D  | Up to 450 Mbps | 6,500,000/month |
| Biznet        | METRONET 5D  | Up to 900 Mbps | 12,500,000/month |
| First Media   | Joy Value    | 50 Mbps       | 340,000/month |
| First Media   | Joy Pro      | 100 Mbps      | 415,000/month |
